# EX05 SolutionsSolutions to the fifth week's exercises on nonlinear optimization and camera calibration.

In [ ]:
import numpy as npfrom numpy.linalg import normfrom scipy.optimize import least_squares# Helper functions for homogeneous coordinatesdef to_homogeneous(p):    return np.append(p,1)def from_homogeneous(p):    return p[:-1]/p[-1]# Linear triangulation using DLTdef triangulate(qs, Ps):    A=[]    for q,P in zip(qs,Ps):        u,v=q        A.append(u*P[2]-P[0])        A.append(v*P[2]-P[1])    A=np.stack(A)    _,_,V=np.linalg.svd(A)    X=V[-1]    return from_homogeneous(X)

In [ ]:
# Camera setupK = np.array([[700,0,600],              [0,700,400],              [0,0,1]],dtype=float)R1 = np.eye(3)R2 = np.eye(3)t1 = np.array([0,0,1.0])t2 = np.array([0,0,20.0])def projection_matrix(K,R,t):    Rt = np.hstack([R,t.reshape(3,1)])    return K @ RtP1 = projection_matrix(K,R1,t1)P2 = projection_matrix(K,R2,t2)Q = np.array([1.0,1.0,0.0])def project(P,Q):    q_h = P @ to_homogeneous(Q)    return from_homogeneous(q_h)q1 = project(P1,Q)q2 = project(P2,Q)print('q1 =',q1)print('q2 =',q2)

### Exercise 5.1The projection matrices are built from the given intrinsics and extrinsics. Projecting $Q=[1,1,0]^T$ gives the image points $q_1$ and $q_2$ above.

In [ ]:
# Exercise 5.2q1_tilde = q1 + np.array([1.0,1.0])q2_tilde = q2 + np.array([1.0,1.0])Q_tilde = triangulate([q1_tilde, q2_tilde],[P1,P2])print('Q_tilde =', Q_tilde)q1_hat = project(P1,Q_tilde)q2_hat = project(P2,Q_tilde)err1 = norm(q1_hat - q1_tilde)err2 = norm(q2_hat - q2_tilde)print('reprojection error camera1:', err1)print('reprojection error camera2:', err2)print('distance from true Q:', norm(Q_tilde - Q))

### Exercise 5.2Adding one pixel of noise to each observation and applying linear triangulation leads to noticeable reprojection error and a biased 3D estimate.

In [ ]:
# Exercise 5.3def triangulate_nonlin(qs, Ps):    qs=[np.asarray(q) for q in qs]    Ps=[np.asarray(P) for P in Ps]    def compute_residuals(X):        Q=X.reshape(3)        res=[]        for q,P in zip(qs,Ps):            q_proj=project(P,Q)            res.extend(q_proj - q)        return np.array(res)    x0=triangulate(qs,Ps)    sol=least_squares(compute_residuals,x0)    return sol.x

### Exercise 5.3`triangulate_nonlin` refines the 3D point by minimizing the reprojection residuals via Levenberg–Marquardt (`least_squares`).

In [ ]:
# Exercise 5.4Q_hat = triangulate_nonlin([q1_tilde, q2_tilde],[P1,P2])print('Q_hat =', Q_hat)q1_hat = project(P1,Q_hat)q2_hat = project(P2,Q_hat)err1 = norm(q1_hat - q1_tilde)err2 = norm(q2_hat - q2_tilde)print('reprojection error camera1:', err1)print('reprojection error camera2:', err2)print('distance from true Q:', norm(Q_hat - Q))

### Exercise 5.4The non-linear refinement yields a smaller reprojection error and a 3D point closer to the ground truth than the linear result.

### Exercise 5.5Capture roughly twenty images of a planar checkerboard from varied orientations. Ensure the target spans the frame and disable any in-camera lens correction.

### Exercise 5.6If some frames are stored in portrait and others in landscape orientation, they correspond to different intrinsic matrices. Mixing them violates the assumption of constant intrinsics and corrupts the calibration.

### Exercise 5.7Use `cv2.findChessboardCorners` on each image. Begin with a downsampled view to determine the correct number of internal corners. Discard images where the checkerboard cannot be detected due to excessive pose or occlusion.

### Exercise 5.8Generate corner coordinates with `checkerboard_points(n,m)` and pair them with the detected image points. Estimate $K$, $R$, and $t$ using `cv2.calibrateCamera` or a custom bundle-adjustment implementation.

### Exercise 5.9Reproject the checkerboard corners using the recovered parameters and compute the root mean square reprojection error for each frame. Visual comparison of detected and reprojected points helps identify outliers.

### Exercise 5.10Form $Q = 2\cdot	ext{box3d}() + 1$ and project these 3D points into one calibrated image to verify the estimated pose. The projected cuboid should align with the scene as in the exercise sheet.

### Exercise 5.11Repeat the calibration while estimating the first radial distortion coefficient $k_1$. A lower reprojection error indicates that distortion is being modeled. Use the estimated parameters to undistort an image if lens distortion is visible.